Example workflow to setup an FTU using APIs, generating executable codes for various experimental designs, and generating code for infering operators from data

In [1]:
import numpy as np
import json
from ftuutils import phsutils

Load the positions at which cells exist

In [3]:
points = None

with open('../examples/data/positions.npy', 'rb') as f:
    earr = np.load(f)
    xarr = np.load(f)
    points = np.load(f)

For this example we will choose a subset of nodes from the above positions (to reduce problem size)

In [4]:
np.random.seed(0)
pids = np.arange(points.shape[0])
np.random.shuffle(pids)
maxpoints = pids.shape[0]//12
print(f"Selecting {maxpoints} of {pids.shape[0]}")
spids = pids[:maxpoints]
points = points[spids,:]

Selecting 5 of 64


Here we construct a graph based on Delaunay triangulation with constraints on maximum edge lengths. 
In cases where nodes and their interconnections are known, construct a Networkx graph and a FTUGraph can be created from the Networkx graph

In [5]:
from ftuutils.base import FTUDelaunayGraph 
#Set edge weights based on orientation with respect to conductivity tensor
conductivity_tensor = np.array([1.2,0.9,0.5]) #Fibre sheet normal
#Parameterised conductivity tensor can also be provided. 
#These values can be set at the time of creating an experiment
conductivity_tensor = np.array(['Df','Ds','Dn'],dtype=str) #Fibre sheet normal
g = FTUDelaunayGraph(points,"APN",conductivity_tensor)

Determine nodes that will be on the boundary i.e. nodes that will communicate with the environment. These nodes need not necessarily be on the physical boundary of the discrete cell network

In [6]:
#Stimulus block is along the left wall
#Find nodes close to x start
left = np.min(points,axis=1)
leftpoints = np.isclose(points[:,0],left[0])
simb = []
#Get the nodes that are on the left edge
for i,v in enumerate(leftpoints):
    if v:
        simb.append(i)       
print("Selected input nodes",simb)

Selected input nodes [4]


Given a graph describing the cellular interconnections and a set of nodes that exchange energy (input nodes), we can start constructing an FTU (specifically the discrete part of it).

The example below uses a modified form of AlievPanfilov Excitation Contraction model that has membrane voltage and active tension as state variables. The model also has an approximate Hamiltonian description.

In [7]:
#Create a dictionary to store connection information
phsdata = {}
#FTUGraph type assigns a network id, we will use this network for membrane current exchange
defaultNetworkId = g.getDefaultNetworkID()
#Specify for each PHS class, for each input component the network on which it connects
phsdata = phsutils.connect(phsdata , 'APN','i_1',defaultNetworkId) #Connection on u - membrane voltage, network weight has conductivity encoded

Boundary connections are specified as below. As a convention, boundary networks are negatively numbered.

Nodes that receive external input are specified as below - these are boundary connections and the network number is negative.
All external inputs with the same network number share the same input variable.


If different input variables are required for each  external input, provide different network numbers for the nodes this may be useful for much finer control of inputs.

In this example we assume that all the input nodes are excited by the same stimulus.

In [8]:
for ein in simb:
    phsdata = phsutils.addExternalInput(phsdata,ein,'i_1',-2)

#Indicate which networks are dissipative and add the information to the phsdata dictionary
networkDissipation = {defaultNetworkId:True}
networkNames = {defaultNetworkId:"ucap",-2:"ubar"}
#The dictionary keys networkNames and networkDissipation are keywords for composition logic and must be adhered
phsdata["networkNames"] = networkNames
phsdata["networkDissipation"] = networkDissipation

Provide Cell type descriptions, these can be constructed by hand, by FTUWeaver or through libbondgraph api.

Below we create a single celltype with the name `APN`

In [9]:
phsval = r'{"parameter_values":{"eps0":{"value":"0.002","units":"dimensionless"},"k":{"value":"8.0","units":"dimensionless"},"a":{"value":"0.13","units":"dimensionless"},"c0":{"value":"0.016602","units":"dimensionless"},"ct":{"value":"0.0775","units":"dimensionless"},"mu1":{"value":"0.2","units":"dimensionless"},"mu2":{"value":"0.3","units":"dimensionless"},"x1":{"value":"0.0001","units":"dimensionless"},"x2":{"value":"0.78","units":"dimensionless"},"x3":{"value":"0.2925","units":"dimensionless"},"eta1":{"value":"Tai*(Heaviside(u-x2)*Heaviside(x3-Tai)*(x1-c0)+c0)","units":"dimensionless"},"eta2":{"value":"Heaviside(u-x2)*Heaviside(x3-Tai)*(x1-c0)+c0","units":"dimensionless"},"sigma":{"value":"0.042969","units":"dimensionless"},"sqpi":{"value":"sqrt(2*3.141459265)","units":"dimensionless"},"kV":{"value":"exp(-0.5*((u-1)/sigma)**2)/(sigma*sqpi)","units":"dimensionless"},"U":{"value":"k*u*(u-a)*(1-u)-u*v","units":"dimensionless"},"V":{"value":"(eps0+(v*mu1)/(u+mu2))*(-v-k*u*(u-a-1))","units":"dimensionless"}},"Hderivatives":{"cols":1,"rows":4,"elements":["Tai","Ta","u","v"]},"hamiltonianLatex":"- Ta c_{0} kV + \\frac{Tai^{2} eta1}{2} - \\frac{eps0 k u^{3}}{3} + \\frac{eps0 k u^{2} \\left(a + 1\\right)}{2} - i_{1} v","hamiltonian":"eps0*k*((a+1)*u**2)/2 - eps0*k*u**3/3 + eta1*Tai**2/2 - c0*kV*Ta - (i_1)*v","portHamiltonianMatrices":{"matJ":{"cols":4,"rows":4,"elements":["0","- eta1/2","c0*kV/2","0","eta1/2","0","0","0","-c0*kV/2","0","0","0","0","0","0","0"]},"matR":{"cols":4,"rows":4,"elements":["c0","-eta1/2","-c0*kV/2","0","-eta1/2","eta2","0","0","-c0*kV/2","0","-U","0","0","0","0","-V"]},"matB":{"cols":4,"rows":4,"elements":["0","0","0","0","0","0","0","0","0","0","1/ct","0","0","0","0","0"]},"matBhat":{"cols":0,"rows":0,"elements":[]},"matQ":{"cols":4,"rows":4,"elements":["1","0","0","0","0","1","0","0","0","0","1","0","0","0","0","1"]},"matE":{"cols":4,"rows":4,"elements":["1","0","0","0","0","1","0","0","0","0","1/ct","0","0","0","0","1/ct"]},"matC":{"cols":0,"rows":0,"elements":[]},"u":{"cols":1,"rows":4,"elements":["0","0","i_1","0"]},"u_connect2boundary":{"cols":1,"elements":[false,false,false,false],"rows":4}},"stateVector":{"cols":1,"rows":4,"elements":["Tai","Ta","u","v"]},"state_values":{"Tai":{"value":0.000,"units":"dimensionless"},"Ta":{"value":0.001,"units":"dimensionless"},"u":{"value":0,"units":"dimensionless"},"v":{"value":0.03604,"units":"dimensionless"}},"isphenomenological":false,"success":true}'
phstypes = {'APN':json.loads(phsval)}

With the above information and a graph with appropriate node and edge attributes, we can compose a FTU.

In [10]:
#Get the graph
G = g.getGraph()

#Call the FTU composition logic to create a FTU with above information

#composer = g.composeCompositePHS(G,phstypes,phsdata)

#The above call will create a composite phs, whose parameters are substituted in the final
#expression. Use this approach when the PHS parameters will not be changed to explored in the experiments
#When experiments with differing phs parameters need to created, the composite PHS
#can be created such that the parameters are not substituted at build time but resolved at runtime
#For such approaches use
composer = g.composeCompositePHS(G,phstypes,phsdata,substituteParameters=False)

In [11]:
import sympy
stateVec = sympy.Matrix(composer.stateVec)
ucapVec = sympy.Matrix([f"u_{s}" for s in composer.stateVec])
Ccap = composer.uyConnectionMatrix
Delx = composer.Qcap * stateVec  # Potential
# Since E^-1 can be expensive, we will scale by the rate diagonal value of E for that component
Einv = sympy.eye(composer.Ecap.shape[0])
for i in range(composer.Ecap.shape[0]):
    Einv[i, i] = 1 / composer.Ecap[i, i]
JRQx = (composer.Jcap - composer.Rcap) * Delx
interioru = composer.Bcap * Ccap * (composer.Bcap.T) * ucapVec
exterioru = composer.Bdas * sympy.Matrix(composer.uVecSymbols).T
rhs = sympy.SparseMatrix(Einv * (JRQx - interioru + exterioru))
rhs

Matrix([
[                                                                                                                                                                                                                              -Tai_1*c0_1 + c0_1*kV_1*u_1],
[                                                                                                                                                                                                                              -Ta_1*eta2_1 + Tai_1*eta1_1],
[                                                                                        ct_1*(U_1*u_1 - 2*u_u_1 + u_u_2*Abs(0.025*Dn - 0.025000000000000005*Ds) + u_u_5*Abs(0.024999999999999998*Df + 0.009150635094610967*Dn - 0.009150635094610966*Ds))],
[                                                                                                                                                                                                                                       

In [12]:
composer.Bdas

Matrix([
[     0],
[     0],
[     0],
[     0],
[     0],
[     0],
[     0],
[     0],
[     0],
[     0],
[     0],
[     0],
[     0],
[     0],
[1/ct_4],
[     0],
[     0],
[     0],
[     0],
[     0]])

In [13]:
composer.exportAsPython()

'\n# The content of this file was generated using the FTUWeaver\n\nfrom enum import Enum\nimport numpy as np\n\n\n__version__ = "0.0.1"\n\nSTATE_COUNT = 20\nVARIABLE_COUNT = 135\n\ndef heaviside(x):\n    if x > 0:\n        return 1.0\n    return 0.0\n    \ndef Abs(x):\n    return np.fabs(x)\n\nclass VariableType(Enum):\n    VARIABLE_OF_INTEGRATION = 0\n    STATE = 1\n    CONSTANT = 2\n    COMPUTED_CONSTANT = 3\n    ALGEBRAIC = 4\n    INTERNAL_INPUT = 5\n    EXTERNAL_INPUT = 6\n\nVOI_INFO = {"name": "t", "units": "second", "component": "main", "type": VariableType.VARIABLE_OF_INTEGRATION}\nSTATE_INFO = [\n\t{"name": "Tai_1", "units": "dimensionless", "component": "main", "type": VariableType.STATE},\n\t{"name": "Ta_1", "units": "dimensionless", "component": "main", "type": VariableType.STATE},\n\t{"name": "u_1", "units": "dimensionless", "component": "main", "type": VariableType.STATE},\n\t{"name": "v_1", "units": "dimensionless", "component": "main", "type": VariableType.STATE},\n\t{"n

In [14]:
rhs[2]

ct_1*(U_1*u_1 - 2*u_u_1 + u_u_2*Abs(0.025*Dn - 0.025000000000000005*Ds) + u_u_5*Abs(0.024999999999999998*Df + 0.009150635094610967*Dn - 0.009150635094610966*Ds))